In [1]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.plotting import lag_plot
import matplotlib
matplotlib.style.use('ggplot')
get_ipython().run_line_magic('matplotlib', 'inline')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

plt.show(block=True)

sns.set(style="darkgrid", rc={'figure.figsize':(30, 10)})

from sklearn import preprocessing

In [2]:
bd_path = './'
bd_fallas_filename = 'bd_falla_summer.csv'
bd_op_filename = 'bd_op_summer.csv'
#bd_mo_filename = ''

df_fallas = pd.read_csv(bd_path + bd_fallas_filename, parse_dates = True)
df_op =  pd.read_csv(bd_path + bd_op_filename, parse_dates = True, index_col = 'time')
#df_mo =  pd.read_csv(bd_path + bd_mo_filename, parse_dates = True, index_col = 'time')

In [3]:
# chequeamos por duplicados en bd_op
print(len(df_op[df_op.index.duplicated(keep = False)].index))

0


In [5]:
df_op.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1578240 entries, 2016-01-01 00:00:00 to 2018-12-31 23:59:00
Columns: 411 entries, ID to estadoCorrea_cv07
dtypes: float64(375), int64(10), object(26)
memory usage: 4.9+ GB


In [6]:
estados = [col for col in df_op.columns if 'estado' in col.lower()]
estados

['estado_motor_cv701',
 'estado_CH1',
 'estado_CH2',
 'estadoPicaRoca_CH1',
 'estadoPicaRoca_CH2',
 'estadoCorrea_cv01',
 'estado_cv05',
 'estadoStockPilePlanta_cv05',
 'estadoCorrea_cv06',
 'estadoCorrea_cv07']

In [7]:
on_off = [col for col in df_op.columns if 'off' in col.lower()]
on_off

['feeder05_onOff',
 'feeder06_onOff',
 'feeder07_onOff',
 'feeder07_onOff.1',
 'feederSpeed_15_onOff',
 'feederSpeed_16_onOff',
 'feederSpeed_17_onOff',
 'feederSpeed_18_onOff',
 'feederSpeed_19_onOff',
 'feederSpeed_20_onOff',
 'feederSpeed_21_onOff',
 'feederSpeed_22_onOff',
 'feederSpeed_701_onOff',
 'feederSpeed_702_onOff',
 'feederSpeed_703_onOff',
 'feederSpeed_704_onOff']

In [31]:
categoricas = [ col  for col, dt in df_op.dtypes.items() if dt == object]
enteras = [col  for col, dt in df_op.dtypes.items() if dt == 'int64']
reales = [col  for col, dt in df_op.dtypes.items() if dt == 'float64']
print(len(categoricas) + len(enteras) + len(reales))


411


In [11]:
df_op[categoricas]

ID estado_motor_cv701 feeder05_onOff feeder06_onOff  \
time                                                                            
2016-01-01 00:00:00        0           DETENIDO       DETENIDO       DETENIDO   
2016-01-01 00:01:00        1           DETENIDO       DETENIDO       DETENIDO   
2016-01-01 00:02:00        2           DETENIDO       DETENIDO       DETENIDO   
2016-01-01 00:03:00        3           DETENIDO       DETENIDO       DETENIDO   
2016-01-01 00:04:00        4           DETENIDO       DETENIDO       DETENIDO   
...                      ...                ...            ...            ...   
2018-12-31 23:55:00  1578235           DETENIDO       DETENIDO    FUNCIONANDO   
2018-12-31 23:56:00  1578236           DETENIDO       DETENIDO       DETENIDO   
2018-12-31 23:57:00  1578237           DETENIDO       DETENIDO       DETENIDO   
2018-12-31 23:58:00  1578238           DETENIDO       DETENIDO       DETENIDO   
2018-12-31 23:59:00  1578239           DETENIDO       DETENIDO       DETENIDO   

                    feeder07_onOff feeder07_onOff.1  stockPile_nivel7  \
time                                                                    
2016-01-01 00:00:00       DETENIDO         DETENIDO                 0   
2016-01-01 00:01:00       DETENIDO         DETENIDO                 0   
2016-01-01 00:02:00       DETENIDO         DETENIDO                 0   
2016-01-01 00:03:00       DETENIDO         DETENIDO                 0   
2016-01-01 00:04:00       DETENIDO         DETENIDO                 0   
...                            ...              ...               ...   
2018-12-31 23:55:00    FUNCIONANDO      FUNCIONANDO                 0   
2018-12-31 23:56:00       DETENIDO         DETENIDO                 0   
2018-12-31 23:57:00       DETENIDO         DETENIDO                 0   
2018-12-31 23:58:00       DETENIDO         DETENIDO                 0   
2018-12-31 23:59:00       DETENIDO         DETENIDO                 0   

                     stockPile_nivel8  stockPile_nivel9  stockPile_nivel10  \
time                                                                         
2016-01-01 00:00:00                 0                 0                  0   
2016-01-01 00:01:00                 0                 0                  0   
2016-01-01 00:02:00                 0                 0                  0   
2016-01-01 00:03:00                 0                 0                  0   
2016-01-01 00:04:00                 0                 0                  0   
...                               ...               ...                ...   
2018-12-31 23:55:00                 0                 0                  0   
2018-12-31 23:56:00                 0                 0                  0   
2018-12-31 23:57:00                 0                 0                  0   
2018-12-31 23:58:00                 0                 0                  0   
2018-12-31 23:59:00                 0                 0                  0   

                     ... estadoPicaRoca_CH2 estadoCorrea_cv01  estado_cv05  \
time                 ...                                                     
2016-01-01 00:00:00  ...           DETENIDO              FUNC     DETENIDO   
2016-01-01 00:01:00  ...           DETENIDO              FUNC     DETENIDO   
2016-01-01 00:02:00  ...           DETENIDO              FUNC     DETENIDO   
2016-01-01 00:03:00  ...           DETENIDO              FUNC     DETENIDO   
2016-01-01 00:04:00  ...           DETENIDO              FUNC     DETENIDO   
...                  ...                ...               ...          ...   
2018-12-31 23:55:00  ...           DETENIDO             DETEN  FUNCIONANDO   
2018-12-31 23:56:00  ...           DETENIDO             DETEN     DETENIDO   
2018-12-31 23:57:00  ...           DETENIDO             DETEN     DETENIDO   
2018-12-31 23:58:00  ...           DETENIDO             DETEN     DETENIDO   
2018-12-31 23:59:00  ...           DETENIDO             DETEN     DETEN

In [14]:
df_op[['stockPile_nivel8','stockPile_nivel7','stockPile_nivel9','stockPile_nivel10']].describe()

stockPile_nivel8  stockPile_nivel7  stockPile_nivel9  stockPile_nivel10
count         1578240.0         1578240.0         1578240.0          1578240.0
mean                0.0               0.0               0.0                0.0
std                 0.0               0.0               0.0                0.0
min                 0.0               0.0               0.0                0.0
25%                 0.0               0.0               0.0                0.0
50%                 0.0               0.0               0.0                0.0
75%                 0.0               0.0               0.0                0.0
max                 0.0               0.0               0.0                0.0

In [15]:
df_op[categoricas].dtypes

ID                             int64
estado_motor_cv701            object
feeder05_onOff                object
feeder06_onOff                object
feeder07_onOff                object
feeder07_onOff.1              object
stockPile_nivel7               int64
stockPile_nivel8               int64
stockPile_nivel9               int64
stockPile_nivel10              int64
feederSpeed_15_onOff          object
feederSpeed_16_onOff          object
feederSpeed_17_onOff          object
feederSpeed_18_onOff          object
feederSpeed_19_onOff          object
feederSpeed_20_onOff          object
feederSpeed_21_onOff          object
feederSpeed_22_onOff          object
feederSpeed_701_onOff         object
feederSpeed_702_onOff         object
feederSpeed_703_onOff         object
feederSpeed_704_onOff         object
feederSpeed_704_pct            int64
estado_CH1                    object
estado_CH2                    object
estadoPicaRoca_CH1            object
estadoPicaRoca_CH2            object
e

In [16]:
df_op['vibracionAfd21_DE_cv07'].describe()

count    1578240.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0
Name: vibracionAfd21_DE_cv07, dtype: float64

In [17]:
df_op['tempAceiteTrafoAfd21_cv07'].describe()

count    1578240.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0
Name: tempAceiteTrafoAfd21_cv07, dtype: float64

In [19]:
df_dict = pd.DataFrame(index = list(df_op.columns))

In [24]:
df_dict["Tipo"] = ""

In [25]:
df_dict.loc[categoricas,"Tipo"] = "text"

In [32]:
df_dict.loc[enteras, "Tipo"] = "int"

In [33]:
df_dict.loc[reales, "Tipo"] = "float"

In [53]:
cv005 = list(df_op.columns[104:225])
cv006 = list(df_op.columns[225:330])
cv007 = list(df_op.columns[330:])


In [44]:
df_dict["equipo"] = ""


121


In [54]:
cv007

['velocidadDriverAfd11_cv07',
 'torqueDriverAfd11_cv07',
 'voltajeLinea1Afd11_cv07',
 'potenciaActivaAfd11_cv07',
 'factorPotenciaAfd11_cv07',
 'vibreacionAfd11_DE_cv07',
 'vibracionAfd11_DN_cv07',
 'voltaje11_cv07',
 'velocidad11_cv07',
 'torque11_cv07',
 'corrienteMotor11_cv07',
 'voltajeMotor11_cv07',
 'corrienteLineaAfd11_cv07',
 'velocidadDriver21_cv07',
 'torqueDriver21_cv07',
 'voltajeLineaAfd21_cv07',
 'potenciaActiva21_cv07',
 'factorPotencia21_cv07',
 'vibracionAfd21_DE_cv07',
 'vibracionAfd21_DN_cv07',
 'voltaje21_cv07',
 'velocidad21_cv07',
 'torque21_cv07',
 'corrienteMotor21_cv07',
 'voltajeMotor21_cv07',
 'corrienteLinea21_cv07',
 'velocidadCorreaSigno_cv07',
 'torqueTotalAfd_cv07',
 'controlVelocidadRampa_cv07',
 'potenciaTotalMotor_cv07',
 'tensionCeldaCargaIzq_cv07',
 'tensionCeldaCargaDer_cv07',
 'tensionCorrea_cv07',
 'velocidad_cv07',
 'velocidadPoleaPrimaria_cv07',
 'velocidadPoleaSecundaria_cv07',
 'tempFreno11_cv07',
 'tempAceiteTrafo11_cv07',
 'tempDescansoAfd1

In [55]:
df_dict.loc[cv005, 'equipo'] = 'cv05'
df_dict.loc[cv006, 'equipo'] = 'cv06'
df_dict.loc[cv007, 'equipo'] = 'cv07'

'estadoCorrea_cv06'

In [57]:
ch1 = [col for col in df_op.columns[1:104] if 'ch1' in col.lower()]
ch2 = [col for col in df_op.columns[1:104] if 'ch2' in col.lower()]
cv001 = [col for col in df_op.columns[1:104] if 'cv01' in col.lower()]
cv701 = [col for col in df_op.columns[1:104] if '701' in col.lower()]

print(len(ch1)+len(ch2)+len(cv001)+len(cv701))

51


In [58]:
resto = [col for col in df_op.columns[1:104] if col not in ch1 and col not in ch2 and col not in cv001 and col not in cv701 ]

In [59]:
resto

['presionHidroset_cr01',
 'alimentacionFeeder_cv05',
 'tonelajePromedio_cv05',
 'inventarioStockPilePlanta',
 'alturaPilaDescarga',
 'velocidadFeeder_05',
 'velocidadFeeder_06',
 'velocidadFeeder_07',
 'velocidadFeeder_08',
 'feeder05_onOff',
 'feeder06_onOff',
 'feeder07_onOff',
 'feeder07_onOff.1',
 'TpH_SAG1',
 'TpH_SAG2',
 'TpH_SAG3',
 'stockPile_nivelSensor1',
 'stockPile_nivelSensor2',
 'stockPile_nivel',
 'stockPile_nivelDesc1',
 'stockPile_nivelDesc2',
 'stockPile_nivelDesc3',
 'stockPile_nivelDesc4',
 'stockPile_nivelDesc5',
 'stockPile_nivelDesc6',
 'stockPile_nivelDesc7',
 'stockPile_nivelDesc8',
 'stockPile_nivel7',
 'stockPile_nivel8',
 'stockPile_nivel9',
 'stockPile_nivel10',
 'feederSpeed_15_onOff',
 'feederSpeed_16_onOff',
 'feederSpeed_17_onOff',
 'feederSpeed_18_onOff',
 'feederSpeed_15_pct',
 'feederSpeed_16_pct',
 'feederSpeed_17_pct',
 'feederSpeed_18_pct',
 'feederSpeed_19_onOff',
 'feederSpeed_20_onOff',
 'feederSpeed_21_onOff',
 'feederSpeed_22_onOff',
 'feeder

In [60]:
df_dict.to_csv("diccionario.csv")

In [61]:
df_op.columns[233:238]

Index(['voltajeMotor12_cv06', 'corrienteMotor12_cv06', 'voltaje12_cv06',
       'vibración_DE_12_cv06', 'vibración_DN_12_cv06'],
      dtype='object')

In [63]:
import unidecode
d = {col: unidecode.unidecode(col) for col in df_op.columns}
df_op.rename(columns = d, inplace=True)

In [64]:
df_op.to_csv('bd_op_summer2.csv')

In [65]:
df_dict.index = [unidecode.unidecode(col) for col in list(df_dict.index)]

In [66]:
df_op.columns[233:238]

Index(['voltajeMotor12_cv06', 'corrienteMotor12_cv06', 'voltaje12_cv06',
       'vibracion_DE_12_cv06', 'vibracion_DN_12_cv06'],
      dtype='object')

In [71]:
columnas = list(df_dict.index)
cols2 = list(df_op.columns)
print(len(columnas))
print(len(cols2))

411
411


In [69]:
columnas

['ID',
 'estado_motor_cv701',
 'velocidad_cv701_rpm',
 'velocidad_cv701',
 'pesometro_cv701',
 'pesometro_cv01',
 'velocidadCero_cv01',
 'velocidadCero_A_cv01',
 'presionHidroset_cr01',
 'potenciaActiva_A_motor_CH1',
 'posicionCono_CH1',
 'corrientePromedio_CH1',
 'T_ambiente_enfriadores_CH1',
 'presionHidroset_cr701',
 'potencia_CH701',
 'posicionCono_CH2',
 'cap_termica_motorCH2',
 'cap_termica_motorCH1',
 'corrientePromedio_CH2',
 'T_ambiente_enfriadores_CH2',
 'potenciaActivaMotor_cv01A',
 'potenciaActivaMotor_cv01B',
 'corrienteMotor_cv01A',
 'corrienteMotor_cv01B',
 'potencia_M1_cv701',
 'potencia_M2_cv701',
 'corrienteMotorA_cv701_faseA',
 'corrienteMotorB_cv701_faseA',
 'corrienteMotorA_cv701_faseB',
 'corrienteMotorB_cv701_faseB',
 'corrienteMotorA_cv701_fasec',
 'corrienteMotorB_cv701_faseC',
 'alimentacionFeeder_cv05',
 'tonelajePromedio_cv05',
 'inventarioStockPilePlanta',
 'alturaPilaDescarga',
 'velocidadFeeder_05',
 'velocidadFeeder_06',
 'velocidadFeeder_07',
 'velocida

In [72]:
for i in range(411):
    c1 = columnas[i]
    c2 = cols2[i]
    if c1 != c2:
        print("problemas: ", i, c1, c2)

In [73]:
df_dict.to_csv('diccionario2.csv')